In [2]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Hate Video task'
assert FOLDERNAME is not None, "[!] Enter the foldername."

Mounted at /content/drive


In [3]:
!pip install noisereduce
import noisereduce as nr
import cv2
import math
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader
from statistics import mode
import moviepy.editor
import librosa
%matplotlib inline
dtype = torch.cuda.FloatTensor
device = torch.device("cuda:0")

/usr/local/lib/python3.6/dist-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
a_model = nn.Sequential(
    nn.Linear(257,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,2),
    nn.Softmax(dim = 1)
).to(device)

In [5]:
def initialize_weights(m):
  if isinstance(m, nn.Linear) or isinstance(m, nn.ConvTranspose2d):
    nn.init.xavier_uniform_(m.weight.data)

initialize_weights(a_model)

In [6]:
def process_audio(audio_data):
  audio_data = nr.reduce_noise(audio_clip=audio_data, noise_clip=audio_data, verbose=False)
  trimmed, index = librosa.effects.trim(audio_data, top_db=20, frame_length=512, hop_length=64)
  stft = np.abs(librosa.stft(trimmed, n_fft=512, hop_length=256, win_length=512))
  stft = np.mean(stft, axis = 1, keepdims = True)
  return stft

In [10]:
X = []
y = []
audio_folder = '/content/drive/My Drive/Hate Video task/Dataset/Training_audio/'
for entry in os.listdir(audio_folder):
    audio_name = audio_folder + entry
    try:
      audio_file, sr = librosa.load(audio_name)
    except:
      continue
    if(entry[0]=='h'):
      y.append(1)
    else:
      y.append(0)
    audio_x = process_audio(audio_file)
    X.append(audio_x)
    del audio_x
X = np.array(X)
y = np.array(y)
print(X.shape)

(241, 257, 1)


In [ ]:
dim = tensor_x.shape
tensor_x = torch.Tensor(X).to(device).type(torch.cuda.FloatTensor)
del X
tensor_y = torch.Tensor(y).to(device).type(torch.cuda.LongTensor)
del y

In [18]:
tensor_x = torch.reshape(tensor_x, (dim[0],dim[1]))


train_fin_dataset = TensorDataset(tensor_x, tensor_y)
train_dataloader = DataLoader(train_fin_dataset, batch_size = 16)

In [ ]:
lr = 0.000001
optimizer = optim.Adam(a_model.parameters(), lr = lr, betas = (0.5, 0.999), weight_decay=1e-5)
loss = nn.CrossEntropyLoss()
net_losses = []
k = np.arange(200)
for i in k:
  net_loss = 0
  for x,b in train_dataloader:
    scores = a_model(x)
    optimizer.zero_grad()
    scores = a_model(x)
    curr_loss = loss(scores,b)
    net_loss += curr_loss
    curr_loss.backward()
    optimizer.step()
  net_losses.append(net_loss)
  print("Epoch {} --- Loss {}".format(i,net_loss))
  if(i%25==24):
    lr /= 5

In [ ]:
range = np.arange(200)
plt.plot(range, net_losses)

In [33]:
path = '/content/drive/My Drive/Hate Video task/Models/'
torch.save(a_model.state_dict(), path+'Model2.pt')